In [1]:
# import modules and define functions
from pyscf import gto, scf, fci
import pandas as pd
import numpy as np
import os
from functools import reduce
from scipy import linalg
os.environ['OMP_NUM_THREADS'] = "1" 

/Users/nickyang/opt/anaconda3/envs/htvs/lib/python3.7/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Setting h5py.default_file_mode is deprecated. 'r' (read-only) is the default from h5py 3.0.
  h5py.get_config().default_file_mode = 'r'


In [2]:
# build H2 with minimal basis
atom = 'O 0 0 0; O 0 0 1.21'
basis = 'sto-3g'
# initiate a triplet O2
o2_triplet = gto.Mole()
o2_triplet.build(atom=atom,basis=basis,charge=0,spin=2)

In [3]:
# calculate UHF and ROHF energies of the triplet O2
o2_0_3_uhf = scf.UHF(o2_triplet).run()
o2_0_3_rohf = scf.ROHF(o2_triplet).run()

converged SCF energy = -147.634048505054  <S^2> = 2.003411  2S+1 = 3.0022732
converged SCF energy = -147.632274661318


In [4]:
# Energy differences
delE_0_3_U_RO = (o2_0_3_uhf.e_tot - o2_0_3_rohf.e_tot) * 627.509
print('The UHF energy is lower than the ROHF energy by %s for O2_0_3' %(round(delE_0_3_U_RO, 2)))

The UHF energy is lower than the ROHF energy by -1.11 for O2_0_3


In [5]:
# S^2 values for various orbitals
print('The <S^2> value for UHF O2_0_3 is %.2f' %(o2_0_3_uhf.spin_square()[0]))
print('The <S^2> value for ROHF O2_0_3 is %.2f' %(o2_0_3_rohf.spin_square()[0]))

The <S^2> value for UHF O2_0_3 is 2.00
The <S^2> value for ROHF O2_0_3 is 2.00


In [6]:
# perform post HF calculations
# perform a CASSCF calculation with 6 active orbitals and 8 active electrons
ncas, nelecas = (6,8)
o2_0_3_casscf = o2_0_3_rohf.CASSCF(ncas, nelecas).run()

CASSCF energy = -147.738218757139
CASCI E = -147.738218757139  E(CI) = -21.0695711517413  S^2 = 2.0000000


In [18]:
# Energy differences
delE_0_3_U_CAS = (o2_0_3_uhf.e_tot - o2_0_3_casscf.e_tot) * 627.509
print('The UHF energy is lower than the CAS energy by %s for O2_0_3' %(round(delE_0_3_U_CAS, 2)))

The UHF energy is lower than the CAS energy by 65.37 for O2_0_3


In [9]:
# do a Full CI run
o2_0_3_fci = fci.FCI(o2_0_3_rohf)
o2_0_3_fci.run()

In [17]:
# Energy differences
delE_0_3_U_FCI = (o2_0_3_uhf.e_tot - o2_0_3_fci.e_tot) * 627.509
print('The UHF energy is lower than the FCI energy by %s for O2_0_3' %(round(delE_0_3_U_FCI, 2)))

The UHF energy is lower than the FCI energy by 69.49 for O2_0_3
